In [2]:
from google.cloud import bigquery

client = bigquery.Client()

dataset_ref = client.dataset("san_francisco", project = "bigquery-public-data")

dataset = client.get_dataset(dataset_ref)

tables = list(client.list_tables(dataset))

for table in tables:
    print(table.table_id)

Using Kaggle's public dataset BigQuery integration.
311_service_requests
bikeshare_stations
bikeshare_status
bikeshare_trips
film_locations
sffd_service_calls
sfpd_incidents
street_trees


In [3]:
table_ref = dataset_ref.table("bikeshare_trips")

table = client.get_table(table_ref)

client.list_rows(table, max_results = 5).to_dataframe()

,trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type
0,1235850,1540,2016-06-11 08:19:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-11 08:45:00+00:00,San Jose Diridon Caltrain Station,2,124,15206,Customer
1,1219337,6324,2016-05-29 12:49:00+00:00,San Jose Diridon Caltrain Station,2,2016-05-29 14:34:00+00:00,San Jose Diridon Caltrain Station,2,174,55416,Customer
2,793762,115572,2015-06-04 09:22:00+00:00,San Jose Diridon Caltrain Station,2,2015-06-05 17:28:00+00:00,San Jose Diridon Caltrain Station,2,190,95391,Customer
3,453845,54120,2014-09-15 16:53:00+00:00,San Jose Diridon Caltrain Station,2,2014-09-16 07:55:00+00:00,San Jose Diridon Caltrain Station,2,127,81,Customer
4,1245113,5018,2016-06-17 20:08:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-17 21:32:00+00:00,San Jose Diridon Caltrain Station,2,153,95070,Customer


# **Task 1:**
# Use an analytic function to calculate the cumulative number of trips for each date in 2015

In [5]:
# Query to count the (cumulative) number of trips per day
# step 1: Aggregate daily trip data by using "CTE" 


num_trips_query = """
                  WITH trips_by_day AS
                  (
                  SELECT DATE(start_date) AS trip_date,
                         COUNT(*) as num_trips
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE EXTRACT(YEAR FROM start_date) = 2015
                  GROUP BY trip_date
                  )
                  SELECT *,
                        SUM(num_trips)
                        OVER(
                            ORDER BY trip_date
                            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                        ) AS cumulative_trips
                  FROM trips_by_day
                  """

num_trips_result = client.query(num_trips_query).result().to_dataframe()
num_trips_result.head()

/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,trip_date,num_trips,cumulative_trips
0,2015-01-01,181,181
1,2015-01-02,428,609
2,2015-01-03,283,892
3,2015-01-04,206,1098
4,2015-01-05,1186,2284



# Task 2:

# Tracks the stations where each bike began (in start_station_id) and ended (in end_station_id) the day on October 25, 2015.


In [10]:
# Query to track beginning and ending stations on October 25, 2015, for each bike
# FIRST_VALUE, LAST_VALUE

start_end_query = """
                  SELECT bike_number,
                         TIME(start_date) AS trip_time,
                         FIRST_VALUE(start_station_id)
                             OVER(
                                 PARTITION BY bike_number
                                 ORDER BY start_date
                                 ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                             ) AS first_station_id,
                         LAST_VALUE(end_station_id)
                               OVER(
                                   PARTITION BY bike_number
                                   ORDER BY start_date
                                   ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                               ) AS last_station_id,
                         start_station_id, end_station_id
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE DATE(start_date) = '2015-10-25'
                  """


start_end_result = client.query(start_end_query).result().to_dataframe()
start_end_result.head()

/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,bike_number,trip_time,first_station_id,last_station_id,start_station_id,end_station_id
0,22,13:25:00,2,16,2,16
1,25,11:43:00,77,51,77,60
2,25,12:14:00,77,51,60,51
3,29,14:59:00,46,74,46,60
4,29,21:23:00,46,74,60,74
